In [1]:
import numpy as np
from matplotlib import pyplot
from geojson.feature import *
import os
from tqdm import tqdm
import h3
import geopandas as gpd
import json
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans
import folium
from folium import Map, Marker, GeoJson
import branca.colormap as cm
from branca.colormap import LinearColormap
import vaex
import pandas as pd
from math import pi, cos, sin
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
from shapely.geometry import Polygon
from folium.features import Choropleth
from branca.colormap import linear
from sklearn.preprocessing import MinMaxScaler

In [2]:
df_cleaned = vaex.open('./data/trips_prepared.hdf5')

In [49]:
df_cleaned.head()

#,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,trip_start_hour,trip_end_hour,trip_start_4h_period,trip_end_4h_period,pickup_hex,dropoff_hex
0,1b545d0be27b195076e5d2a7d4fc0ebef6ef20ac,'56a1119c6ca57e39525cf06829f9ecff553cf4b5ac24821...,2017-01-01 00:00:00.000000,2017-01-01 00:00:00.000000,240,1.3,1.70318e+10,1.70318e+10,32,28,6.25,0,0,0,6.25,Cash,Taxi Affiliation Services,41.881,-87.6327,41.8853,-87.6572,0,0,0,0,872664c1affffff,872664cadffffff
1,c3ee40051c5643c54064a8b9df3686d2d411518d,'7987cfe5e3b2c9869665f70fe35dbe7423e7fc5852f2d6f...,2017-01-01 00:00:00.000000,2017-01-01 00:15:00.000000,1500,0.4,1.70318e+10,1.70311e+10,32,8,22.75,4.85,0,1.5,29.1,Credit Card,Blue Ribbon Taxi Association Inc.,41.881,-87.6327,41.8983,-87.6208,0,0,0,0,872664c1affffff,872664c1effffff
2,29f30c8a6bbc028f5d63aa18c708d31b47267a9f,'0b269137bd94b55212311226d9e1554423e12bcc16a432c...,2017-01-01 00:00:00.000000,2017-01-01 00:30:00.000000,1620,0.9,1.70318e+10,1.70311e+10,32,8,12.25,2.85,0,2,17.1,Credit Card,Taxi Affiliation Services,41.881,-87.6327,41.8925,-87.6262,0,0,0,0,872664c1affffff,872664c1effffff
3,55ae4fa955e6bc8705b995248c34f4183d848c27,'d1d781b6807176ed53fbb9d0010dd0a87e29c073984bda6...,2017-01-01 00:00:00.000000,2017-01-01 00:00:00.000000,540,1,1.70311e+10,1.70311e+10,8,8,6.75,0,0,2,8.75,Cash,Dispatch Taxi Affiliation,41.9028,-87.6261,41.8909,-87.6189,0,0,0,0,872664c1effffff,872664c1effffff
4,ce908cd441e28d08dfb22bf168102beabb5e16e4,'f7630edc68a131c4a97382dd93206949da80ec1443d8da7...,2017-01-01 00:00:00.000000,2017-01-01 00:15:00.000000,480,0.5,1.70311e+10,1.70311e+10,8,8,6.25,2,0,1.5,9.75,Credit Card,Taxi Affiliation Services,41.8925,-87.6262,41.8909,-87.6189,0,0,0,0,872664c1effffff,872664c1effffff
5,fd25560e18dab95225999c264586a331dc3283f5,'9ff153898c75dbd960a207b660a71f92f79b666dcec98e1...,2017-01-01 00:00:00.000000,2017-01-01 00:30:00.000000,1980,3.4,1.70318e+10,1.70311e+10,32,8,16.75,3.65,0,1.5,21.9,Credit Card,Taxi Affiliation Services,41.881,-87.6327,41.895,-87.6197,0,0,0,0,872664c1affffff,872664c1effffff
6,2504e0bf058ff310ac4c197618a843f88fc1c0c6,'d3a5f1452b4a1db2a547e7a0aabfe28d67c62b0e40155fa...,2017-01-01 00:00:00.000000,2017-01-01 00:15:00.000000,960,5.9,1.70313e+10,1.70311e+10,32,6,17.5,0,0,1,18.5,Cash,Northwest Management LLC,41.8774,-87.622,41.9427,-87.6518,0,0,0,0,872664c1affffff,872664c16ffffff
7,4ca1087133dfd8bde2483cb2aaa609fff2654f7c,'0083fcde0fb490b4ec424c63d1d750378f6ad11154d1d4a...,2017-01-01 00:00:00.000000,2017-01-01 00:15:00.000000,780,0.1,1.70312e+10,1.70311e+10,24,8,9.5,2.1,0,1,12.6,Credit Card,Taxi Affiliation Services,41.8997,-87.665,41.892,-87.6319,0,0,0,0,872664cadffffff,872664c1effffff
8,04bc3089a41c33afc9b6cbf5a9beac4fed91da7c,'785ef52cff80fe78a7ee899453036c0be6ac8fef4d10068...,2017-01-01 00:00:00.000000,2017-01-01 00:00:00.000000,120,0.4,1.70318e+10,1.70318e+10,32,32,4.25,2,0,1,7.25,Credit Card,Chicago Medallion Leasing INC,41.881,-87.6327,41.881,-87.6327,0,0,0,0,872664c1affffff,872664c1affffff
9,e09945909ae154d6b0ce433468baddcf79f79973,'b329626db30d23db0e6620de7d1a3036dd3c2ca3e139fd4...,2017-01-01 00:00:00.000000,2017-01-01 00:00:00.000000,180,0.7,1.70318e+10,1.70318e+10,32,28,5,0,0,2,7,Cash,Dispatch Taxi Affiliation,41.881,-87.6327,41.8853,-87.6572,0,0,0,0,872664c1affffff,872664cadffffff


In [51]:
df_compromised = df_cleaned['dropoff_hex','trip_seconds', 'trip_miles']

In [3]:
resolution = 7

In [4]:
def convert_to_h3(lat,long):
    return h3.geo_to_h3(lat, long, resolution)

In [5]:
df_cleaned['pickup_hex'] = df_cleaned.apply(convert_to_h3, [df_cleaned['pickup_centroid_latitude'], df_cleaned['pickup_centroid_longitude']])

In [6]:
df_cleaned['dropoff_hex'] = df_cleaned.apply(convert_to_h3, [df_cleaned['dropoff_centroid_latitude'], df_cleaned['dropoff_centroid_longitude']])

In [ ]:
df_cleaned = df_cleaned.groupby(by='dropoff_hex').agg({'Count': vaex.agg.count('dropoff_hex')}) 
df_cleaned.sort(by='Count', ascending= False)

In [7]:
def h3_to_poly(row):
    points = h3.h3_to_geo_boundary(
        row, geo_json=True)
    return Polygon(points)

In [ ]:
df_cleaned["Geometry"] = df_cleaned["dropoff_hex"].apply(
    h3_to_poly)

In [ ]:
df_cleaned = df_cleaned.to_pandas_df()
df_cleaned['Count'] = df_cleaned['Count'].astype("float")

In [ ]:
legend_drop = linear.YlOrRd_09.scale(min(df_cleaned['Count']), max(df_cleaned['Count']))

In [ ]:
df_cleaned['Count_scaled'] = np.log1p(df_cleaned['Count'])

In [ ]:
cm_drop = linear.YlOrRd_09.scale(min(df_cleaned['Count_scaled']), max(df_cleaned['Count_scaled']))

In [ ]:
m = folium.Map(location=[41.8781, -87.6298], zoom_start=11)

list_features = []

for i in range(1, len(df_cleaned)):
    count = df_cleaned.iloc[i]['Count']
    feature = Feature(geometry= df_cleaned.iloc[i]['Geometry'],
                      id = df_cleaned.iloc[i]['dropoff_hex'],
                      properties = {"resolution": resolution, "count": count })
    list_features.append(feature)


feat_collection = FeatureCollection(list_features)
geojson_result = json.dumps(feat_collection)


GeoJson(
        geojson_result,
        style_function = lambda feature: {
           #cm_drop(float(feature['properties']['count'])),
            'color': cm_drop(float(feature['properties']['count'])),
            'weight': 2,
            'fillOpacity': 0.5,
            'fillColor': cm_drop(float(feature['properties']['count'])),
        },
        name = "Example"
    ).add_to(m)
legend_drop.add_to(m)

In [ ]:
m

In [ ]:
df_cleaned.head()

In [21]:
df_time = df_cleaned['pickup_hex', 'dropoff_hex', 'trip_seconds', 'trip_miles']

In [22]:
#time per mile is expressed in miles per second
df_time['time_per_mile'] = df_time['trip_miles'] / df_time['trip_seconds']

In [10]:
df_time.head()

Process SpawnPoolWorker-7:
Process SpawnPoolWorker-8:
Process SpawnPoolWorker-6:
Process SpawnPoolWorker-4:
Process SpawnPoolWorker-3:
Process SpawnPoolWorker-5:
Process SpawnPoolWorker-1:
Process SpawnPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/vincent/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/vincent/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/vincent/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/vincent/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/vincent/opt/anaconda3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File

#,pickup_hex,dropoff_hex,trip_seconds,trip_miles,time_per_mile
0,872664c1affffff,872664cadffffff,240,1.3,0.00541667
1,872664c1affffff,872664c1effffff,1500,0.4,0.000266667
2,872664c1affffff,872664c1effffff,1620,0.9,0.000555556
3,872664c1effffff,872664c1effffff,540,1,0.00185185
4,872664c1effffff,872664c1effffff,480,0.5,0.00104167
5,872664c1affffff,872664c1effffff,1980,3.4,0.00171717
6,872664c1affffff,872664c16ffffff,960,5.9,0.00614583
7,872664cadffffff,872664c1effffff,780,0.1,0.000128205
8,872664c1affffff,872664c1affffff,120,0.4,0.00333333
9,872664c1affffff,872664cadffffff,180,0.7,0.00388889


In [24]:
df_time = df_time['dropoff_hex', 'time_per_mile']

In [27]:
df_time = df_time.groupby(by='dropoff_hex').agg({'Avg_time_per_mile': vaex.agg.mean('Avg_time_per_mile')}) 
df_time.sort(by='Avg_time_per_mile', ascending= False)

#,dropoff_hex,Avg_time_per_mile
0,872664ca2ffffff,0.01248265918892055
1,87275934effffff,0.010034830232119962
2,872664cccffffff,0.009737280963759457
3,872664ca0ffffff,0.009450609598429964
4,872664c8bffffff,0.009259259259259259
...,...,...
103,872664cadffffff,0.003271797003412384
104,872664ccbffffff,0.003121376811594203
105,872664c88ffffff,0.002698412698412698
106,872664cdeffffff,0.00101056268797771


In [30]:
df_time['Geometry'] = df_time['dropoff_hex'].apply(h3_to_poly)

In [31]:
df_time.head()

#,dropoff_hex,Avg_time_per_mile,Geometry
0,872664ca8ffffff,0.00407085,"'<POLYGON ((-87.69 41.899, -87.702 41.891, -87.6..."
1,872664d88ffffff,0.00657911,"'<POLYGON ((-87.676 41.997, -87.689 41.988, -87...."
2,872664d83ffffff,0.00587944,"'<POLYGON ((-87.718 42.029, -87.731 42.02, -87.7..."
3,872664c86ffffff,0.00525067,"'<POLYGON ((-87.791 41.922, -87.803 41.914, -87...."
4,872664cc8ffffff,0.00476913,"'<POLYGON ((-87.59 41.741, -87.603 41.733, -87.5..."
5,872664d8cffffff,0.00586006,"'<POLYGON ((-87.667 42.017, -87.68 42.009, -87.6..."
6,872664cf0ffffff,0.00522097,"'<POLYGON ((-87.656 41.846, -87.669 41.838, -87...."
7,872664ca4ffffff,0.00576825,"'<POLYGON ((-87.714 41.972, -87.727 41.963, -87...."
8,872664cabffffff,0.00434091,"'<POLYGON ((-87.698 41.878, -87.711 41.87, -87.7..."
9,8726641b2ffffff,0.00623669,"'<POLYGON ((-87.54 41.73, -87.553 41.722, -87.54..."


In [41]:
df2 = df_time.to_pandas_df()
#df_cleaned['time_per_mile'] = df_cleaned['time_per_mile'].astype("float")

In [42]:
df2.head()

,dropoff_hex,Avg_time_per_mile,Geometry
0,872664ca8ffffff,0.004071,POLYGON ((-87.68960859632769 41.89878991031961...
1,872664d88ffffff,0.006579,POLYGON ((-87.67608158738066 41.99655455296839...
2,872664d83ffffff,0.005879,POLYGON ((-87.71807315668339 42.02864962011406...
3,872664c86ffffff,0.005251,POLYGON ((-87.79063432740574 41.92202849176213...
4,872664cc8ffffff,0.004769,POLYGON ((-87.59011429862058 41.74143783571094...


In [43]:
df2['Avg_time_per_mile'] = df2['Avg_time_per_mile'].astype("float")

In [44]:
cm = linear.YlOrRd_09.scale(min(df2['Avg_time_per_mile']), max(df2['Avg_time_per_mile']))

In [47]:
m = folium.Map(location=[41.8781, -87.6298], zoom_start=11)

list_features = []

for i in range(1, len(df2)):
    Avg_time_per_mile = df2.iloc[i]['Avg_time_per_mile']
    feature = Feature(geometry= df2.iloc[i]['Geometry'],
                      id = df2.iloc[i]['dropoff_hex'],
                      properties = {"resolution": resolution, "avg_time": Avg_time_per_mile })
    list_features.append(feature)


feat_collection = FeatureCollection(list_features)
geojson_result = json.dumps(feat_collection)


GeoJson(
        geojson_result,
        style_function = lambda feature: {
            'color': cm(float(feature['properties']['avg_time'])),
            'weight': 2,
            'fillOpacity': 0.5,
            'fillColor': cm(float(feature['properties']['avg_time'])),
        },
        name = "Example"
    ).add_to(m)
#legend_drop.add_to(m)

## Dropoff points with the highest average Time per Mile

In [48]:
m

## Calculating idle time of each trip

In [52]:
merged_df = df_compromised.merge(df_time, on='dropoff_hex', how='left')

AttributeError: 'DataFrameLocal' object has no attribute 'merge'

In [53]:
merged_df = df_compromised.join(df_time, on='dropoff_hex', how='left')

In [54]:
merged_df.head()

#,dropoff_hex,trip_seconds,trip_miles,Avg_time_per_mile,Geometry
0,872664cadffffff,240,1.3,0.0032718,"'<POLYGON ((-87.66 41.903, -87.673 41.895, -87.6..."
1,872664c1effffff,1500,0.4,0.00363846,"'<POLYGON ((-87.63 41.908, -87.643 41.899, -87.6..."
2,872664c1effffff,1620,0.9,0.00363846,"'<POLYGON ((-87.63 41.908, -87.643 41.899, -87.6..."
3,872664c1effffff,540,1,0.00363846,"'<POLYGON ((-87.63 41.908, -87.643 41.899, -87.6..."
4,872664c1effffff,480,0.5,0.00363846,"'<POLYGON ((-87.63 41.908, -87.643 41.899, -87.6..."
5,872664c1effffff,1980,3.4,0.00363846,"'<POLYGON ((-87.63 41.908, -87.643 41.899, -87.6..."
6,872664c16ffffff,960,5.9,0.00496658,"'<POLYGON ((-87.664 41.96, -87.676 41.952, -87.6..."
7,872664c1effffff,780,0.1,0.00363846,"'<POLYGON ((-87.63 41.908, -87.643 41.899, -87.6..."
8,872664c1affffff,120,0.4,0.00349071,"'<POLYGON ((-87.639 41.887, -87.652 41.879, -87...."
9,872664cadffffff,180,0.7,0.0032718,"'<POLYGON ((-87.66 41.903, -87.673 41.895, -87.6..."


In [ ]:
# wieder time per mile für jeden trip bestimmen 
# von jeder time per mile, avg time per mile subtrahieren => = expected time - actual time
# groupen, nach avg. unterschied zum durchschnitt 
# Auf map anzeigen